In [ ]:
import tensorflow as tf
import keras
import os,shutil
import numpy as np 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras.models import Model,Sequential
from keras.layers import Dense,MaxPool2D,Conv2D,Dropout,ZeroPadding2D,Flatten,Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import optimizers
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#run only once
!unzip '/content/drive/My Drive/23079_29550_bundle_archive.zip'

In [ ]:
TRAIN_DIR=r'/content/asl_alphabet_train/asl_alphabet_train'
TEST_DIR=r'/content/asl_alphabet_test/asl_alphabet_test'

In [ ]:
for i in os.listdir(TRAIN_DIR):
  print(i,len(os.listdir(TRAIN_DIR+'/'+i)))

In [ ]:
#run only once
os.mkdir('/content/train2')
os.mkdir('/content/test2')
for cat in os.listdir(TRAIN_DIR):
  os.mkdir('train2/'+cat)
  os.mkdir('test2/'+cat)

In [ ]:

#run only once
for cat in os.listdir(TRAIN_DIR):
  n = len(os.listdir(TRAIN_DIR + '/' + cat))
  bpoint=n//8
  t=np.arange(1,n+1)
  np.random.shuffle(t)
  ttest = t[:bpoint]
  ttrain = t[bpoint:]
  for x in ttest:
    shutil.copy(TRAIN_DIR+'/'+cat+'/'+cat + str(x) + '.jpg' , '/content/test2/'+cat+'/'+cat+str(x)+'.jpg')
  for x in ttrain:
    shutil.copy(TRAIN_DIR+'/'+cat+'/'+cat + str(x) + '.jpg' , '/content/train2/'+cat+'/'+cat+str(x)+'.jpg')

In [ ]:
train = '/content/train2'
test =  '/content/test2'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,brightness_range=[0.3,1.4],width_shift_range=0.2,height_shift_range=0.2,)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train,target_size=(200,200),batch_size=128,shuffle=True,class_mode='categorical')
test_set = test_datagen.flow_from_directory(test,target_size=(200,200),batch_size=128,shuffle=True,class_mode='categorical')


Found 76125 images belonging to 29 classes.
Found 10875 images belonging to 29 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), activation = 'relu', input_shape = (200, 200, 3)),
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(29, activation = 'softmax'),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 198, 198, 8)       224       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 99, 99, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 97, 97, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 46, 46, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 64)       

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])
history = model.fit(training_set,epochs=3,validation_data=test_set)

Epoch 1/3
595/595 [==============================] - 125s 210ms/step - loss: 0.8511 - accuracy: 0.7421 - val_loss: 0.1384 - val_accuracy: 0.9574
Epoch 2/3
595/595 [==============================] - 123s 207ms/step - loss: 0.0836 - accuracy: 0.9729 - val_loss: 0.0495 - val_accuracy: 0.9850
Epoch 3/3
595/595 [==============================] - 123s 207ms/step - loss: 0.0388 - accuracy: 0.9880 - val_loss: 0.0346 - val_accuracy: 0.9909


In [ ]:
dictrev = {}
for i in test_set.class_indices:
    dictrev[test_set.class_indices[i]] = i

dictrev

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z',
 26: 'del',
 27: 'nothing',
 28: 'space'}

In [ ]:
for img in os.listdir(TEST_DIR):
  temp = image.load_img(TEST_DIR+'/'+img)
  temp = image.img_to_array(temp)
  temp = np.expand_dims(temp,axis=0)
  res = model.predict(temp)
  print(img,dictrev[res.argmax()])

Q_test.jpg Q
P_test.jpg P
T_test.jpg T
I_test.jpg I
N_test.jpg N
J_test.jpg J
V_test.jpg V
S_test.jpg S
space_test.jpg space
F_test.jpg F
L_test.jpg L
Y_test.jpg Y
Z_test.jpg Z
H_test.jpg H
C_test.jpg C
R_test.jpg R
U_test.jpg U
D_test.jpg D
W_test.jpg W
G_test.jpg G
A_test.jpg A
K_test.jpg K
M_test.jpg X
nothing_test.jpg nothing
E_test.jpg E
B_test.jpg B
X_test.jpg X
O_test.jpg O


In [ ]:
model.save('asl_model1.h5')